In [11]:
## Loading Libraries
import pandas as pd
import numpy as np
import sys
import jdatetime as jd
import os
import glob
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [12]:
%%time
## Loading DATA
os.chdir(r"D:\Shareholder Data\Cleaned - Merged with Price")
all_filenames = [i for i in glob.glob('*.{}'.format('csv'))]
SDATA = pd.concat([pd.read_csv(f,index_col=0).drop_duplicates() for f in all_filenames ])

## Marketcap Data
os.chdir(r"D:\Shareholder Data/")
mcap= pd.read_csv('mcap.csv',index_col=0)
# Conver date from string to jdatetime
mcap['True_Date'] = pd.to_datetime(mcap['True_Date'], format='%Y-%m-%d')
G = mcap.True_Date.drop_duplicates()
J = G.apply(lambda x: jd.date.fromgregorian(day=x.day,month=x.month,year=x.year))
DataOrg_date_GtoJ_dict = dict(zip(G,J))
mcap['Jalali_Date']=mcap.True_Date.map(DataOrg_date_GtoJ_dict)

Wall time: 2min 5s


In [13]:
# Mergind with Marketcap Data
SDATA.rename(columns={'MarketCap':'MarketCap_'},inplace=True)
SDATA = pd.merge(SDATA,mcap[['Symbol','Jalali_Date','MarketCap']],left_on=['Symbol','Jalali_Date'],
                 right_on=['Symbol','Jalali_Date'],how='left')
SDATA.MarketCap_ = SDATA.MarketCap_.fillna(SDATA.MarketCap)
SDATA.rename(columns={'MarketCap_':'MarketCap'},inplace=True)

In [14]:
%%time
# Conver date from string to jdatetime
SDATA['True_Date'] = pd.to_datetime(SDATA['True_Date'], format='%Y-%m-%d')
G = SDATA.True_Date.drop_duplicates()
J = G.apply(lambda x: jd.date.fromgregorian(day=x.day,month=x.month,year=x.year))
DataOrg_date_GtoJ_dict = dict(zip(G,J))
SDATA['Jalali_Date']=SDATA.True_Date.map(DataOrg_date_GtoJ_dict)

Wall time: 17.1 s


In [16]:
 SDATA[SDATA.year==1386]

,Id_tse,Symbol,Jalali_Date,Industry,ShareHolder,percent,quantity,ShareHolder_Type,Shares,MarketCap,...,close,Unadjusted_close,Industry_Id,True_Date,year,month,day,Fill_Flag,chnk_id,MarketCap


In [17]:
# Filtering and keeping only the last date in each year.
for year in list(np.arange(1387,1400,1)):
    print(year)
    temp = SDATA[SDATA.year==year]
    last_day = temp.Jalali_Date.drop_duplicates().sort_values(ascending=True).iloc[-1]
    print('*',last_day)
    df = temp[temp.Jalali_Date == last_day]
    # Saving the file
    os.chdir(r"C:\Users\Mahdi\OneDrive\Master Thesis\Data")
    name = 'Shareholder'+str(year)+'.csv'
    df.to_csv(name)

1387
* 1387-12-28
1388
* 1388-12-26
1389
* 1389-12-28
1390
* 1390-12-28
1391
* 1391-12-28
1392
* 1392-12-28
1393
* 1393-12-27
1394
* 1394-12-26
1395
* 1395-12-28
1396
* 1396-12-28
1397
* 1397-12-28
1398
* 1398-12-28
1399
* 1399-04-01


In [ ]:
# Filtering and keeping only the last date in each month.
for year in list(np.arange(1387,1400,1)):
    print(year)
    temp = SDATA[SDATA.year==year]
    
    for month in range(1,13):
        temp_month = temp[temp.month==month]
        
        try:
            last_day = temp_month.Jalali_Date.drop_duplicates().sort_values(ascending=True).iloc[-1]
            df = temp_month[temp_month.Jalali_Date == last_day]
            # Saving the file
            os.chdir(r"C:\Users\Mahdi\OneDrive\Master Thesis\Data\MonthlyShareholder")
            name = 'Shareholder'+str(year)+'_'+str(month)+'.csv'
            df.to_csv(name)
        except:
            print('!'+str(year)+'_'+str(month)+'!')

In [18]:
temp = SDATA[['Symbol','year']].drop_duplicates()
print('Number of Firms in each year:')
temp.groupby('year').agg('count')

year=1398
print('Number of Firms in each year-month for year', year,':')
temp = SDATA[SDATA.year==year][['month','Symbol']]
temp = temp.drop_duplicates()
temp.groupby('month').agg('count')


print('Number of Obs in each year:')
SDATA[['year','Symbol']].groupby('year').agg('count')

Number of Firms in each year:


,Symbol
year,
1387,240
1388,247
1389,274
1390,304
1391,313
1392,330
1393,372
1394,387
1395,424


Number of Firms in each year-month for year 1398 :


,Symbol
month,
1,618
2,619
3,621
4,631
5,617
6,614
7,631
8,633
9,640


Number of Obs in each year:


,Symbol
year,
1387,180628
1388,647620
1389,320142
1390,352214
1391,378231
1392,376253
1393,400125
1394,437207
1395,479481
